# Information technologies
Выполнил Смирнов Богдан Викторович

Группа ИС/б-23-1-о

## Особенности работы с изменяемыми типами данных

Для начала я хотел бы обсудить некоторые тонкости при работе со списками, словарями и другими
изменяемыми типами данных. Рассмотрим простой пример вызова функции.


In [ ]:
def my_func(x): # инициализация функции
 x = x + 1 # сложение аргумента с 1
 return x # возврат значения

Эта функция увеличивает свой аргумент на единицу и возвращает то, что получилось.


In [ ]:
y = 5 # инициализиция переменной
print("Function returns", my_func(y)) # вывод значения функции
print("y =", y) # вывод самой переменной

Function returns 6
y = 5


Ничего неожиданного. Функция не изменила переменную y и не должна была этого делать. Давайте
теперь попробуем сделать что-то аналогичное со списком.


In [ ]:
def other_func(my_list): # инициализируем функцию
 my_list.append(1) # добавляем значение
 return my_list # возвращаем список

Эта функция получает в качестве аргумента список, добавляет к нему в конец элемент 1 и
возвращает то, что получилось. Попробуем её вызвать:

In [ ]:
this_list = [6, 9, 33] # инициализируем список
print("this_list before function:",this_list) # выводим список до функции
print("function returns:",other_func(this_list)) # выводим список во время исполнения функции
print("this_list after function", this_list) # выводим список после исполнения функции

this_list before function: [6, 9, 33]
function returns: [6, 9, 33, 1]
this_list after function [6, 9, 33, 1]


Ой. Произошло что-то странное. Функция other_func модифицировала список, который ей был
передан, хотя создан он был вне этой функций и внутри функции не был определён как global (да и
вообще, внутри функции работа велась с другой переменной).
Почему это произошло? Чтобы разобрать в этом вопросе, проще всего посмотреть на визуализацию.

In [ ]:
%load_ext tutormagic # запускаем в jupiter tutormagic

In [ ]:
%%tutor --lang python3 # добавляем к блоку
def other_func(my_list): # инициализируем функцию
 my_list.append(1) # добавляем значение
 return my_list # возвращаем список
this_list = [6, 9, 33] # инициализируем список
other_func(this_list) # смотрим что произойдёт


## Две сортировки

Давайте рассмотрим ещё два примера, которые показывают различие между этими двумя сценариями
— в них снова будут участвовать списки. Выполните две визуализации и найдите, в чём разница.
После этого читайте дальше.

In [ ]:
%%tutor --lang python3 # добавляем к блоку
def return_sorted(my_list): # инициализируем функцию
 my_list = sorted(my_list) # сортируем и присваиваем новой переменной
 return my_list # возвращаем новую переменную
this_list = [33, 1, 55] # инициализируем список
print("this_list before function:", this_list) # вывод до
print("function returned:", return_sorted(this_list)) # вывод во время
print("this_list after function:", this_list) # вывод после

In [ ]:
%%tutor --lang python3 # добавляем к блоку
def sort_and_return(my_list): # инициализируем функцию 
 my_list.sort() # сортируем уже существующую переменную (блок данных в heap)
 return my_list # возвращаем переменную
this_list = [33, 1, 55]  # инициализируем список
print("this_list before function:", this_list) # вывод до
print("function returned:", sort_and_return(this_list)) # вывод во время
print("this_list after function:", this_list) # вывод после

Итак, в чём разница?

Функции return_sorted() и sort_and_return() выполняют примерно одну и ту же работу:
принимают на вход список, сортируют его и возвращают. В тот момент, когда происходит вход в
функцию (Step 5), ситуация в обоих фрагментах идентична: вне функции есть переменная
this_list , внутри функции есть переменная my_list , они обе указывают на один и тот же список.
Ключевое различие происходит на следующем шаге. Функция return_sorted() использует функцию
sorted() , создающую новый список. Затем она присваивает ( = ) результат выполнения sorted()
переменной my_list . Это приводит к тому, что переменная с этим именем начинает ссылаться на
новый объект (Step 7), а старый (на который по-прежнему ссылается this_list ) остаётся в
неприкосновенности. Совсем иначе действует sort_and_return() — она использует метод sort() ,
сортирующий список in place, внутри самого себя. При этом новый объект не создаётся, операция
присваивания не используется и переменная my_list продолжает ссылаться на тот же список, что и
раньше. Просто он оказывается отсортированным.

Обратите внимание на схожесть return_sorted() и my_func() из примера выше: в обоих случаях
используется оператор присваивания. Разница состоит в том, что с числовой переменной никак иначе
поступить было нельзя, поскольку числа неизменяемы, а в случае со списком у разработчика есть
выбор: можно создать новый объект и присвоить его старой переменной, а можно изменить уже
существующий объект, не создавая новый.

## Создание копии


Предположим теперь, что мы хотим написать функцию, которая принимает на вход список,
возвращает такой же список, но с одним добавленным элементом, а сам исходный список не меняет.
Это можно сделать, например, вот так:

In [ ]:
def return_append(L, a): # инициализируем функцию
 new_L = L.copy() # копируем L в новую переменную
 new_L.append(a) # добавляем a
 return new_L # выдаём новый список

In [ ]:
outer_list = [7, 8, 9] # инициализируем список
print("outer_list before funciton", outer_list) # выводим список до
print("function returned", return_append(outer_list, 55)) # выводим список во время
print("outer_list after function", outer_list) # выводим список после

outer_list before funciton [7, 8, 9]
function returned [7, 8, 9, 55]
outer_list after function [7, 8, 9]


Здесь главная хитрость в использовании L.copy() — напомним, что этот метод создаёт копию
существующего списка. Дальше мы снова выполняем операцию присваивания (то есть теперь new_L
является именем для копии списка L , а не для самого списка L ) и мы можем делать с этим новым
списком new_L что угодно. Старый список не изменится.

## Не только вызовы функций

Проблемы, аналогичные рассмотренным выше, возникают не только при вызове функций. Начнём с
простого примера с циклом.


In [ ]:
some_list = [7, 9, 11] # инициализируем список
for x in some_list: # проходимся по списку
 x = x + 1 # прибавляем к x единицу
print(some_list)

[7, 9, 11]


Список some_list не поменялся, и в этом нет ничего удивительного. Но давайте теперь рассмотрим
чуть более сложную ситуацию со списком списков.


In [ ]:
table = [[1, 5], [7, 9]] # инициализируем сложный объект
for row in table: # проходка по его строкам
 row.append(77) # добавление в строку 77
print(table) # вывод

[[1, 5, 77], [7, 9, 77]]


И снова «ой». Что случилось? Посмотрим на визуализаторе.

In [ ]:
%%tutor --lang python3
table = [[1, 5], [7, 9]] # инициализируем сложный объект
for row in table: # проходка по его строкам
 row.append(77) # добавление в строку 77
print(table) # вывод

При выполнении первого шага цикла (Step 3) в переменную row записывается первый элемент
списока table . Однако, этот элемент сам является списком — а точнее ссылкой на список. На
следующем шаге (Step 4) к этому списку добавляется элемент 77 . Потом row становится ссылкой
на второй элемент списка table . К ней тоже добавляется элемент 77 .
Обратите внимание на параллель с предыдущим сюжетом: здесь тоже участвует вызов метода списка,
который меняет этот список in place.

## Головоломка


Как вы думаете, что будет, если выполнить следующий код? Попробуйте его выполнить, посмотрите
на результат и попытайтесь объяснить.


In [ ]:
A = [[]]*5 # инициализируем список из элемента индексом 0 повторяющимся 5 раз? 
A[0].append(1) # добавление в список индекса 0 единицу 
print(A) # печать

[[1], [1], [1], [1], [1]]


## Изменение итерируемого объекта в цикле

В примере выше мы меняли содержимое «внутренних» списков, но сам список table оставался
неизменным: в нём не менялось число элементов и элементы оставались ссылками на те же самые
списки-строки, что и раньше. А можно ли менять сам список во время итераций? Оказывается, можно.
Хотя в большинстве случаев лучше этого не делать. Прежде, чем рассматривать пример, напомним о
том, как работает метод pop() у списка:

In [ ]:
L = [6, 9, 44, 8] # инициализируем список
print(L.pop()) # выкалываем последний элемент
print(L) # выводим список

8
[6, 9, 44]


Он удаляет последний элемент из списка и возвращает его же. Применим теперь его следующим
образом:

In [ ]:
L = [7, 8, 9, 10] # инициализируем список
for x in L: # пробегаемся по списку
 print("Pop element", L.pop()) # выкалываем последний элемент
 print(x) # выводим элемент на котором остановился x

Pop element 10
7
Pop element 9
8


Цикл исполняется два раза: к тому моменту, как цикл закончит обрабатывать элемент 8, элементы 9 и
10 из списка будут уже удалены, необработанных элементов в списке не останется и цикл
прекратится.

Догадаться о том, что произойдёт, можно лишь очень внимательно изучив код. Это значит, что код не
очень хорош: глядя на хороший код, вы можете понять, что он будет делать.

Со словарями ситуация иная.

In [18]:
d = {1:2, 3:4}
for k, v in d.items():
 del d[3]
 print(k, v)

1 2


RuntimeError: dictionary changed size during iteration

Здесь команда del d[3] удаляет элемент с ключом 3 из словаря. Поскольку порядок итерирования
элементов словаря не определён, как корректно продолжить итерации после того, как размер словаря
был изменён, никто не знает. Поэтому такая операция запрещена.

Однако, это не означает, что запрещено менять значение словаря при выполнении цикла. Например,
мы хотим прибавить ко всем значениям число 1. Следующий наивный метод ожидаемо не сработает:

In [22]:
d={1:2, 3:4}
for k, v in d.items():
 v = v + 1
print(d)

{1: 2, 3: 4}


На самом деле, эту задачу следует решать так:

In [21]:
d = {1:2, 3:4}
for k in d:
 d[k] = d[k] + 1
print(d)

{1: 3, 3: 5}


## Множества


Ещё один базовый тип данных в Python — это множество. Оно соответствует математическому
понятию множества — то есть набора каких-то элементов. Каждый элемент может или входить в
множество, или не входить.


> Если вы в данный момент являетесь слушателем нашего курса по Python, то вы
принадлежите множеству слушателей. Нельзя быть «дважды слушателем курса»:
каждый элемент может входить в множество только один раз.


In [23]:
my_set = {6, 9, 11, 11, 9, 'hello'}

In [24]:
my_set


{11, 6, 9, 'hello'}

Как видно из этого простого примера, элементы множества также не упорядочены.


In [25]:
{6, 9, 11, 11, 9, 'hello'} == {9, 'hello', 11, 6}


True

Вот так можно проверить, лежит ли элемент в множестве.


In [26]:
9 in my_set

True

In [27]:
10 in my_set

False

Конечно, оператор in работает не только для множеств. Например, 4 in [2, 4, 8, 10] вернёт
True . Однако для списков эта операция является медленной — вернее, массовой: чем больше
список, тем больше операций сравнения нужно произвести, чтобы понять, лежит ли в нём какой-то
конкретный элемент. В случае с множествами время на проверки практически не растёт с ростом
числа элементов множества.

С множествами можно делать разные операции — с ними мы знакомы по математическим курсам.
Например, объединение и пересечение двух множеств даёт новое множество:

In [31]:
{6, 8, 9} | {6, 11, 7}

{6, 7, 8, 9, 11}

In [32]:
{6, 8, 9} & {6, 11, 7}

{6}

> Отметим ещё раз: порядок элементов во множестве не определён. Если вам нужно
вывести элементы множества в каком-то заранее заданном порядке, то можно
превратить его в сортированный список с помощью функции sorted() .


In [33]:
s = {"Hello", "World", "Test", "Guest", "Aaaaa", "Zzzzz","Zz","Q"}
print(s)
print(sorted(s)) # здесь выводится уже не множество, а список: обратите внимание на квадратные скобки

{'Aaaaa', 'Guest', 'Zzzzz', 'Q', 'Test', 'Zz', 'Hello', 'World'}
['Aaaaa', 'Guest', 'Hello', 'Q', 'Test', 'World', 'Zz', 'Zzzzz']


## Пример использования множеств

допустим, мы просим пользователя ввести команду, но хотим дать ему возможность ввести одну и ту
же команду разными способами. Например, чтобы остановить программу, пользователь может ввести
слово stop или STOP или Stop или просто букву s или S . Можно обработать этот случай с
помощью нескольких условий, соединённых or :


In [35]:
s = 'stop'
if s == 'stop' or s == 'Stop' or s == 'STOP' or s == 'S' or s == 's':
 print("Okay, stopping")

Okay, stopping


А можно создать множество для всех возможных вариаций команды stop и проверять, входит ли
наша команда в это множество:

In [36]:
s = 'stop'
STOPS = {'stop', 'Stop', 'STOP', 'S', 's'}
if s in STOPS:
 print("Okay, stopping") 


Okay, stopping


Впрочем, в этом месте, вероятно, вместо множества можно было бы использовать и просто список.


## Ещё немного про строки


Я давно собирался рассказать про методы для работы со строками. Вообще этих методов существует
много и про все не расскажу, но некоторые из них мы сейчас обсудим.


In [37]:
s = "hello world, hello"
new_s = s.replace("hello", "Hi")
print(new_s)
print(s)

Hi world, Hi
hello world, hello


Вот так, например, можно заменить подстроку в строке. Обратите внимание: строка является
неизменяемым типом данных, поэтому, в отличие от методов списков типа append() методы строк
никогда не меняют саму строку (это вообще невозможно), а вместо этого создают новую строку и
возвращают результат.

Если вы хотели заменить только несколько первых вхождений (например, только первое слово
hello , но не второе), можно было добавить третий аргумент метода replace — он показывает,
сколько раз нужно произвести замену.


In [38]:
"hello world, hello".replace("hello", "Hi", 1)

'Hi world, hello'

Вот так можно найти подстроку в строке:


In [39]:
s.index("world")

6

In [40]:
s.find("world")

6

Оба метода возвращают индекс первого символа подстроки. Разница состоит в том, что если
index() не сможет найти подстроку вообще, он выдаст ошибку (exception), а если с аналогичной
проблемой столкнётся find() , то он вернёт число -1 в качестве индекса.

Кстати, проверить, входит ли подстрока в строку, можно ещё вот так:

In [42]:
"world" in s

True

А вот так можно посчитать, сколько подстрок в строке встречается:


In [43]:
s.count("o")


3

Подробности можно найти в официциальной справке (https://docs.python.org/3/library/string.html).


## Файловый ввод-вывод

Мы начинаем работать с файлами. Сейчас будем обсуждать только чтение и запись. О том, как
запускать файлы на исполнение, отдельная история — для этого существует метод subprocess , мы
до него когда-нибудь дойдём. (Может быть.) Также для начала речь пойдёт о текстовых файлах или
похожих на текстовые (например, код на Python или CSV-файл будет текстовым). Бывают также
бинарные файлы, которые «глазками» читать бесполезно — о некоторых из них будет отдельный
рассказ.

Допустим, мы хотим прочитать файл.


In [44]:
f = open("func.txt")
s = f.read()
f.close()
print(s)

FileNotFoundError: [Errno 2] No such file or directory: 'func.txt'

Что здесь произошло? Во-первых, мы открыли для чтения файл func.txt , лежащий в нашем
текущем рабочем каталоге. Узнать, какой каталог рабочий, можно следующим образом:


In [45]:
import os
os.getcwd()

'd:\\Projects\\Python-labs'

Функция open() вернула объект типа file — переменную, которую можно использовать, чтобы
работать с файлом. Затем мы считали содержимое файла в строчку s , после чего закрыли файл.
Закрывать файлы очень полезно: если вы забудете закрыть файл, другое приложение не сможет его
открыть (например, чтобы в него что-нибудь записать).

Функция read() считает весь файл в одну большую строковую переменную. Это не всегда бывает
удобно (учитывая, что строки в Python неизменяемые и из-за этого работа с ними не всегда бывает
эффективной), поэтому есть разные другие сценарии работы с файлами. Например, можно считать
содержимое файла в список, разбив по строкам.

In [50]:
f = open("func.txt")
lines = f.readlines()
f.close()

In [51]:
print(lines)

['Hello world\n', 'I Love Python\n', 'File ends here ->']


Заметим, что каждая из строк заканивается символом перевода строки \n — они присутствовали в
файле и мы их честно из него считали. Вот так можно вывести файл по строкам, пронумеровав их:

In [52]:
for i, line in enumerate(lines, 1):
 print(i, line, end="")

1 Hello world
2 I Love Python
3 File ends here ->

Другой способ это сделать — не создавать отдельный список, а итерировать прямо сразу файловый
объект.


In [53]:
f = open("func.txt")
for i, line in enumerate(f, 1):
 print(i, line, end="")
f.close()

1 Hello world
2 I Love Python
3 File ends here ->

Этот метод является более предпочтительным, если файл большой. В этом случае считать его в
память целиком может быть невозможно, а обработать по одной строчке — вполне возможно.

Здесь есть, правда, некоторые хитрости. Рассмотрим, например, такой код:


In [54]:
f = open("func.txt")
for line in f:
 print(line, end="")
print("----The next one----")
for line in f:
 print(line, end="")
f.close()


Hello world
I Love Python
File ends here ->----The next one----


Что здесь произошло? Почему второй цикл вообще не выполнился (ничего не выведено после строчки
----The next one---- )? Очень просто: переменная f , хоть и прикидывается списком строк, когда
мы её итерируем, на самом деле таковым не является. В действительности при открытии файла мы
запоминаем позицию, на которой мы этот файл читаем. Изначально она указывает на самое начало
файла, но с каждой итерацией сдвигается. Когда мы прочитаем файл целиком, дальнейшие попытки
из него что-то прочитать ни к чему не приведут: указатель текущей позиции сдвинулся до самого конца
и файл закончился.

Впрочем, есть возможность вернуться в начало: для этого нужно использовать метод seek() .

In [55]:
f = open("func.txt")
for line in f:
 print(line, end="")
print("----The next one----")
f.seek(0)
for line in f:
 print(line, end="")
f.close()

Hello world
I Love Python
File ends here ->----The next one----
Hello world
I Love Python
File ends here ->

## Запись в файлы

Чтобы содать файл и записать в него что-то, нужно открыть его на запись. Это делается путём
передачи функции open второго аргумента — здесь надо написать строчку "w" (от write).


> __Внимание!__ Если файл, который вы пытаетесь открыть на запись, уже существует, он
будет __удалён без какого-либо предупреждения__.


Записывать информацию в файл, открытый на запись, можно, например, с помощью метода
write() .


In [56]:
f = open("other.txt", "w")
f.write("Hello\n")
f.close()

Проверим, что получилось:

In [58]:
open('other.txt').read()

C:\Users\Bogdan\AppData\Local\Temp\ipykernel_24080\3501507054.py:1: ResourceWarning: unclosed file <_io.TextIOWrapper name='other.txt' mode='r' encoding='cp1251'>
  open('other.txt').read()


'Hello\n'

Мы видим, что действительно записали в файл other.txt строчку Hello\n . Заметим, что здесь мы
открыли файл на запись, но не стали присваивать файловый объект какой-либо переменной, а сразу
вызвали от него метод read() . В этом случае файл будет закрыт автоматически через некоторое
время после выполнения этой команды. (Система выдаёт предупреждение о том, что мы не закрыли
файл явно — в некоторых случаях это может приводить к каким-то проблемам.)

## Задачи к блокноту №7


1. Создайте список чисел и измените значение первого элемента на 10.

In [59]:
def replaceFirst(they_list):
    they_list[0] = 10

my_list = [23, 34, 15, 54, 0]
print(my_list)
replaceFirst(my_list)
print(my_list)

[23, 34, 15, 54, 0]
[10, 34, 15, 54, 0]


2. Создайте словарь со значениями по умолчанию и измените значение одного из ключей.

In [63]:
def replaceKey(they_dict):
    they_dict[list(they_dict.keys())[0]] = "muhehe"

my_dict = {1 : "oh", 2 : "no", 3 : "my dictionary", 4 : "it's broken"}
print(my_dict)
replaceKey(my_dict)
print(my_dict)

{1: 'oh', 2: 'no', 3: 'my dictionary', 4: "it's broken"}
{1: 'muhehe', 2: 'no', 3: 'my dictionary', 4: "it's broken"}


3. Создайте список чисел и измените значение второго элемента на 20, затем создайте копию данного списка.


In [66]:
def replaceSecond(they_list):
    they_list[1] = 20
    return they_list.copy()

my_list = [23, 34, 15, 54, 0]
print(my_list)
new_list = replaceSecond(my_list)
print(my_list)
print(new_list)

[23, 34, 15, 54, 0]
[23, 20, 15, 54, 0]
[23, 20, 15, 54, 0]


4. Создайте список чисел и измените значение каждого числа на
сумму его соседей (с учётом самого числа).

In [69]:
def replaceWithSumOfHisFriends(they_list):
    search_list = they_list.copy()
    for i in range(len(they_list)):
        if i == 0:
            they_list[i] += search_list[i + 1]
        elif i == len(they_list) - 1:
            they_list[i] += search_list[i - 1]
        else:
            they_list[i] += search_list[i - 1] + search_list[i + 1]

my_list = [23, 34, 15, 54, 0]
print(my_list)
replaceWithSumOfHisFriends(my_list)
print(my_list)

[23, 34, 15, 54, 0]
[57, 72, 103, 69, 54]


5. Создайте строку и замените все гласные буквы на звёздочки.

6. Создайте строку и замените все согласные буквы на знаки восклицания.


7. Создайте два множества: первое - с элементами {1, 2, 3, 4},
второе - с элементами {3, 4, 5, 6}. Найдите объединение этих множеств и
выведите результат.

8. Создайте два множества: одно с элементами {1, 2, 3}, другое - с
элементами {3, 4, 5}. Найдите симметрическую разность этих множеств и
выведите результат.

9. Создайте два множества: одно с элементами {1, 2, 3, 4, 5}, другое - с элементами {2, 3, 4}. Проверьте, является ли второе множество
подмножеством первого, и выведите результат.

10. Создайте два множества: одно с элементами {1, 2, 3, 4, 5}, другое - с элементами {4, 5, 6, 7, 8}. Найдите пересечение этих множеств и запишите
результат в файл.